__Example 7.15.__(Smith) Consider the spring model

$\ddot{z} + C\dot{z} + Kz = 0$

$z(0) = 2$, $\dot{z}(0) = -C$

with displacement observations so that

$y = \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} z \\ \dot{z} \end{bmatrix} = z$.

The solution is 

$z(t) = 2e^{-Ct/2}\cos{\left( \sqrt{K-C^2/4} t\right)}$.

Suppose that $C$ is unknown. $q=C$ and $p=1$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import stats
import numpy as np

In [ ]:
K = 20.5
C0 = 1.5
sigma = 0.1

In [ ]:
def f(t,C,K):
    # Solution at time t for parameters C and K
    return 2*np.exp(-C*t/2)*np.cos(np.sqrt(K-C**2/4)*t)

Generate "observations" $y$ and plot them

In [ ]:
n = 501 #number of observations
p = 1
t = np.linspace(0, 5, n) #
fig, ax  = plt.subplots()
ax.plot(t, f(t,C0,K));
y = f(t,C0,K) + np.random.normal(0, sigma, n)
ax.plot(t, y,'ro',markersize=1)
plt.xlabel('Time'),plt.ylabel('$z(t)$')
plt.legend(['Truth','Observations $y$'])

In [ ]:
#def SSE(C):
#    return sum( (2*np.exp(-C*t/2)*np.cos(np.sqrt(K-C**2/4)*t)-data)**2 )

Define the sum squared error function and minimize it

In [ ]:
def SSE(C,y):
    return sum( (f(t,C,K)-y)**2 )

In [ ]:
from scipy import optimize
C_OLS=optimize.fmin(SSE,1,args=(y,))
C_OLS

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t, f(t,C_OLS,K)-y,'.');
plt.title('Residuals $\hat\epsilon$')

In [ ]:
sigma_hat = np.sqrt(SSE(C_OLS,y)/(n-p))
sigma_hat


Closed form for the elements of the sensitivity matrix $X$.
$$\dfrac{\partial y}{\partial C} = e^{-C t/2} \left[ \dfrac{Ct}{\sqrt{4K-C^2}} \sin \left( t \sqrt{K - C^2/4}\right) - t \cos \left( t \sqrt{K - C^2/4}\right) \right]$$

In [ ]:
C = C_OLS
w = np.sqrt(K - C**2/4)
dydC = np.exp(-C*t/2)*(C*t/np.sqrt(4*K-C**2)*np.sin(w*t)+t*np.cos(w*t))

$V = \sigma_C^2 = \sigma_0^2 [ \chi(q)^T \chi(q) ]^{-1}$, $\chi(q) = \left[ \dfrac{\partial y}{\partial C} (t_1,q),\dots,\dfrac{\partial y}{\partial C} (t_N,q) \right]^T$

In [ ]:
V = sigma_hat**2/np.dot(dydC,dydC)
np.sqrt(V)

Compute sensitivity matrix by finite differences.

In [ ]:
delta_C = 0.01
X_fd =  (f(t,C_OLS+delta_C,K) - f(t,C_OLS,K))/delta_C
V_fd = sigma_hat**2/np.dot(X_fd,X_fd)
np.sqrt(V_fd)

95% confidence interval is:

In [ ]:
alpha = 0.05
Interval_normal = C_OLS + stats.norm.ppf(1-alpha)*np.sqrt(V)*np.array([-1,1])
Interval_t = C_OLS + stats.t.ppf(1-alpha,df=n-1)*np.sqrt(V)*np.array([-1,1])
Interval_t, Interval_normal

Note that the _interval_ is random. How often is the true value in the interval?
If we repeat the problem (not possible in practice), we can check how often the true value is within the interval.

In [ ]:
mc = 1000
Csamples = np.zeros(mc)
InInterval = np.zeros(mc)
for imc in range(0,mc):
    ysample = f(t,C0,K) + np.random.normal(0, sigma, n)
    Csamples[imc]=optimize.fmin(SSE,1.5,args=(ysample,),disp=0)
    C = Csamples[imc]
    sigma_hat = np.sqrt(SSE(C,ysample)/(n-p))
    w = np.sqrt(K - C**2/4)
    dydC = np.exp(-C*t/2)*(C*t/np.sqrt(4*K-C**2)*np.sin(w*t)+t*np.cos(w*t))
    V = sigma_hat**2/np.dot(dydC,dydC)
    InInterval[imc] = (C0 < C+1.96*np.sqrt(V)) & (C0 > C-1.96*np.sqrt(V))

In [ ]:
np.mean(InInterval)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(Csamples);

In [ ]:
fig, ax  = plt.subplots()
plt.hist(Csamples, color = 'blue', edgecolor = 'black',bins = int(180/5))

In [ ]:
np.mean(Csamples)

In [ ]:
np.std(Csamples),np.sqrt(V)